In [21]:
DEBUG = 1

In [13]:
from selenium import webdriver

import time 

import os

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome(options = options)

driver_aux = webdriver.Chrome(options = options)

In [6]:
url = "https://mangalivre.net/manga/isekai-kenja-no-tensei-musou-geemu-no-chishiki-de-isekai-saikyou/11065"

In [7]:
images = []

In [8]:
driver.get(url)

In [9]:
chapters_list = driver.find_element_by_xpath('//*[@id="chapter-list"]/div[2]/ul')

In [10]:
chapters = chapters_list.find_elements_by_tag_name('li')

In [12]:
for chapter in chapters[::-1]:
    driver_aux.get(chapter.find_element_by_tag_name('a').get_attribute('href'))
    next_page_button = driver_aux.find_element_by_xpath('//*[@id="reader-wrapper"]/div[9]/div[2]/div/div[3]')
    total_pages = int(driver_aux.find_element_by_xpath('//*[@id="reader-wrapper"]/div[9]/div[2]/div/div[2]/span/em[2]').text)
    for i in range(total_pages-1):
        image = driver_aux.find_element_by_xpath('//*[@id="reader-wrapper"]/div[4]/div[2]/div/img').get_attribute('src')
        images.append(image)
        next_page_button.click()
        time.sleep(0.5)
    if DEBUG == 1:
        print(images)

['https://cdn.optimages.net/firefox/Gir12M8CCEIZHwG7Xe8y5g/m5734428/11065/245968/255584/creditos.jpg', 'https://cdn.optimages.net/firefox/Mji3FTxv25LYOgpqEvekAQ/m5734454/11065/245968/255584/pg-1.jpg', 'https://cdn.optimages.net/firefox/w0ELr4n3TmCDnGWIu4BxIw/m5734453/11065/245968/255584/pg-2.jpg', 'https://cdn.optimages.net/firefox/oV3G0SBXTHPLq5-xZUWQ3w/m5734451/11065/245968/255584/pg-3.jpg', 'https://cdn.optimages.net/firefox/W6xb2sqyKKROFyobvMrMng/m5734419/11065/245968/255584/pg-4.png', 'https://cdn.optimages.net/firefox/oo_3dee5RBmIQvROEcPZLw/m5734434/11065/245968/255584/pg-5.png', 'https://cdn.optimages.net/firefox/vbOh2Jx7uxHbCGvXLGV0QA/m5734437/11065/245968/255584/pg-6.png', 'https://cdn.optimages.net/firefox/7hX-sDmIbSFY39BSzenJUg/m5734441/11065/245968/255584/pg-7.png', 'https://cdn.optimages.net/firefox/ySmKX6YkUsyJz4ruxTK-8w/m5734440/11065/245968/255584/pg-8.png', 'https://cdn.optimages.net/firefox/BRIyJgvZQDIlWrwWTALmYQ/m5734417/11065/245968/255584/pg-9.5.jpg', 'https://cdn.